In [49]:
import pandas as pd
accessKey = pd.read_csv('./noty.csv').loc[0].values[1]
secretKey = pd.read_csv('./noty.csv').loc[1].values[1]
projectId = pd.read_csv('./noty.csv').loc[2].values[1]
campaignId = pd.read_csv('./noty.csv').loc[3].values[1]


In [50]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import requests
import json
import pandas as pd
from datetime import datetime, timedelta, timezone

# 설정 정보
config = {
    "accessKey": accessKey,
    "secretKey": secretKey,
    "projectId": projectId,
    "campaignId": campaignId
}

# API 엔드포인트
API_BASE_URL = "https://api.notifly.tech"
AUTH_ENDPOINT = "/authenticate"

# 날짜 범위 설정 (현재 시간 기준 30일 전부터 현재까지)
end_time = datetime.now(timezone.utc)
start_time = end_time - timedelta(days=30)

# ISO 8601 형식으로 변환
start_iso = start_time.strftime('%Y-%m-%dT%H:%M:%S.000Z')
end_iso = end_time.strftime('%Y-%m-%dT%H:%M:%S.999Z')

# 사용자 프로퍼티 파라미터
USER_PARAMS = ["user_id", "member_no"]

# 메트릭 정보 (설명 포함)
METRICS = {
    "send_success": "발송 성공",
    "send_failure": "발송 실패",
    "rendering_failure": "개인화 메시지 렌더링 실패",
    "pending": "발송 시도(집계 중)",
    "push_delivered": "푸시알림 수신",
    "push_click": "푸시알림 클릭",
    "failover_text_message_send_success": "대체 문자 메시지 발송 성공",
    "failover_text_message_send_failure": "대체 문자 메시지 발송 실패",
    "email_send": "이메일 발송",
    "email_delivery": "이메일 도달",
    "email_open": "이메일 오픈",
    "email_click": "이메일 클릭",
    "email_bounce": "이메일 반송",
    "email_complaint": "이메일 수신 거부",
    "email_rendering_failure": "이메일 렌더링 실패",
    "in_app_message_show": "앱 팝업 노출",
    "in_web_message_show": "웹 팝업 노출",
    "main_button_click": "팝업 메인버튼 클릭",
    "close_button_click": "팝업 닫기 버튼 클릭",
    "hide_in_app_message_button_click": "인앱 팝업 다시(특정 기간) 보지 않기 버튼 클릭",
    "cancelled": "취소 이벤트로 인한 발송 취소",
    "skipped__noisy_filter": "중복 발송으로 인한 발송 취소",
    "skipped__fatigue_management": "피로도 관리로 인한 발송 취소",
    "skipped__forbidden_timing_filter": "금지된 발송 시간으로 인한 발송 취소",
    "skipped__excluded_user_filter": "발신 제외 유저로 인한 발송 취소",
    "skipped__aborted_message": "메시지 Abort로 인한 발송 취소(Connected Contents)",
    "skipped__global_frequency_limit_filter": "발송량 제한 조건으로 인한 발송 취소"
}

# 통계 조회 엔드포인트 (쿼리 파라미터 포함)
STATS_ENDPOINT = f"/projects/{config['projectId']}/campaigns/{config['campaignId']}/stats?start={start_iso}&end={end_iso}"


def authenticate():
    """인증 토큰을 가져오는 함수"""
    try:
        print(f"인증 요청 URL: {API_BASE_URL}{AUTH_ENDPOINT}")
        print(f"인증 요청 데이터: {{'accessKey': '{config['accessKey'][:5]}...', 'secretKey': '***'}}")
        
        response = requests.post(
            f"{API_BASE_URL}{AUTH_ENDPOINT}",
            headers={"Content-Type": "application/json"},
            json={
                "accessKey": config["accessKey"],
                "secretKey": config["secretKey"]
            }
        )
        
        print(f"인증 응답 상태 코드: {response.status_code}")
        
        if response.text:
            try:
                response_data = response.json()
                
                if "token" in response_data:
                    return response_data["token"]
                elif "data" in response_data:
                    if isinstance(response_data["data"], str) and response_data["error"] is None:
                        return response_data["data"]
                    elif isinstance(response_data["data"], dict) and "token" in response_data["data"]:
                        return response_data["data"]["token"]
                elif "accessToken" in response_data:
                    return response_data["accessToken"]
                else:
                    print(f"응답에서 토큰을 찾을 수 없습니다.")
                    raise ValueError("응답에서 토큰을 찾을 수 없습니다.")
            except json.JSONDecodeError:
                print(f"인증 응답 본문(텍스트): {response.text[:200]}...")
                raise ValueError("JSON 응답을 파싱할 수 없습니다.")
        else:
            print("인증 응답 본문이 비어 있습니다.")
            raise ValueError("응답 본문이 비어 있습니다.")
        
        response.raise_for_status()
    
    except requests.exceptions.RequestException as e:
        print(f"인증 과정에서 오류 발생: {e}")
        raise


def get_campaign_stats(auth_token, dimensions=None):
    """캠페인 통계를 조회하는 함수"""
    if not auth_token:
        raise ValueError("유효한 인증 토큰이 없습니다.")
    
    endpoint = STATS_ENDPOINT
    if dimensions:
        dimension_params = "&".join([f"dimensions={dim}" for dim in dimensions])
        if "?" in endpoint:
            endpoint = f"{endpoint}&{dimension_params}"
        else:
            endpoint = f"{endpoint}?{dimension_params}"
        
    try:
        print(f"요청 URL: {API_BASE_URL}{endpoint}")
        
        token_display = auth_token[:10] + "..." if len(auth_token) > 10 else auth_token
        print(f"요청 헤더: {{'Authorization': '{token_display}'}}")
        
        response = requests.get(
            f"{API_BASE_URL}{endpoint}",
            headers={
                "Authorization": auth_token,
                "Content-Type": "application/json",
                "Accept": "application/json"
            }
        )
        
        print(f"응답 상태 코드: {response.status_code}")
        
        response.raise_for_status()
        return response.json()
    
    except requests.exceptions.RequestException as e:
        print(f"통계 조회 과정에서 오류 발생: {e}")
        raise


def create_metrics_dataframe(stats_data):
    """캠페인 통계 데이터로부터 데이터프레임 생성
    
    Args:
        stats_data (list): 통계 데이터 목록
        
    Returns:
        pandas.DataFrame: date를 행으로, 모든 가능한 metric을 열로 구성한 데이터프레임
    """
    # 날짜별로 모든 메트릭 데이터를 수집할 딕셔너리
    date_metrics = {}
    
    # 각 메트릭의 일별 데이터 처리
    for metric in stats_data:
        metric_name = metric["metricName"]
        
        for day in metric["dailyCounts"]:
            date = day["date"]
            count = day["count"]
            
            # 해당 날짜의 딕셔너리 초기화 (없는 경우)
            if date not in date_metrics:
                date_metrics[date] = {}
            
            # 해당 날짜에 메트릭 값 추가
            date_metrics[date][metric_name] = count
    
    # 딕셔너리를 데이터프레임으로 변환
    rows = []
    for date, metrics in date_metrics.items():
        row = {"date": date}
        
        # 모든 가능한 메트릭 열 추가 (METRICS 딕셔너리에 있는 모든 키)
        for metric_name in METRICS.keys():
            row[metric_name] = metrics.get(metric_name, 0)
            
        rows.append(row)
    
    # 데이터프레임 생성
    df = pd.DataFrame(rows)
    
    # 날짜로 정렬
    if not df.empty and 'date' in df.columns:
        df = df.sort_values('date')
    
    # date 열이 없는 경우 빈 데이터프레임을 만들고 필요한 열 추가
    if df.empty:
        df = pd.DataFrame(columns=["date"] + list(METRICS.keys()))
    
    return df


def main():
    """메인 실행 함수"""
    try:
        print("Notifly API 클라이언트 시작...")
        print(f"시작 날짜: {start_iso}")
        print(f"종료 날짜: {end_iso}")
        
        # 1. 인증 토큰 가져오기
        auth_token = authenticate()
        print("인증 성공! 토큰 발급 완료")
        
        # 2. 캠페인 통계 조회
        campaign_stats = get_campaign_stats(auth_token, dimensions=USER_PARAMS)
        
        # 3. 데이터프레임 생성
        if "data" in campaign_stats and "stats" in campaign_stats["data"]:
            stats = campaign_stats["data"]["stats"]
            df = create_metrics_dataframe(stats)
            
            # 데이터프레임 저장 (선택사항)
            csv_filename = f"notifly_metrics_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
            df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
            
            return df
        else:
            return pd.DataFrame(columns=["date"] + list(METRICS.keys()))
    
    except Exception as e:
        print(f"실행 중 오류 발생: {e}")
        import traceback
        traceback.print_exc()
        return pd.DataFrame(columns=["date"] + list(METRICS.keys()))


if __name__ == "__main__":
    result_df = main()
    print("\n최종 데이터프레임:")
    print(result_df)

Notifly API 클라이언트 시작...
시작 날짜: 2025-01-28T07:29:43.000Z
종료 날짜: 2025-02-27T07:29:43.999Z
인증 요청 URL: https://api.notifly.tech/authenticate
인증 요청 데이터: {'accessKey': ' fe11...', 'secretKey': '***'}
인증 응답 상태 코드: 400
인증 과정에서 오류 발생: 400 Client Error: Bad Request for url: https://api.notifly.tech/authenticate
실행 중 오류 발생: 400 Client Error: Bad Request for url: https://api.notifly.tech/authenticate

최종 데이터프레임:
Empty DataFrame
Columns: [date, send_success, send_failure, rendering_failure, pending, push_delivered, push_click, failover_text_message_send_success, failover_text_message_send_failure, email_send, email_delivery, email_open, email_click, email_bounce, email_complaint, email_rendering_failure, in_app_message_show, in_web_message_show, main_button_click, close_button_click, hide_in_app_message_button_click, cancelled, skipped__noisy_filter, skipped__fatigue_management, skipped__forbidden_timing_filter, skipped__excluded_user_filter, skipped__aborted_message, skipped__global_frequency_limi

Traceback (most recent call last):
  File "<ipython-input-50-e657d4e1d681>", line 212, in main
    auth_token = authenticate()
  File "<ipython-input-50-e657d4e1d681>", line 107, in authenticate
    response.raise_for_status()
  File "c:\Users\Owner\miniconda3\envs\pymc\lib\site-packages\requests\models.py", line 1021, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 400 Client Error: Bad Request for url: https://api.notifly.tech/authenticate


In [31]:
result_df

,date,send_success,send_failure,rendering_failure,pending,push_delivered,push_click,failover_text_message_send_success,failover_text_message_send_failure,email_send,...,main_button_click,close_button_click,hide_in_app_message_button_click,cancelled,skipped__noisy_filter,skipped__fatigue_management,skipped__forbidden_timing_filter,skipped__excluded_user_filter,skipped__aborted_message,skipped__global_frequency_limit_filter
0,2025-01-28,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2025-01-29,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2025-01-30,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2025-01-31,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2025-02-01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2025-02-02,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,2025-02-03,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,2025-02-04,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,2025-02-05,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,2025-02-06,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
